In [16]:
for unroll in range(1, 8+1, 1):
    with open(f"./small_dgemv_naive_c_implementation_ver0-1_Naive-Unroll-LDA-{unroll}.c", "w") as f:
        # Normal ---------------------------------------------------------------------------------
        remain = ""
        if unroll>=2:
            remain = f"""
        for (int64_t i = lda-i_remain; i<lda; i++) {{
            y[i] += a[i + k*lda] * tmp_x;
        }}        
            """

        unroll_op = ""
        for j in range(0, unroll, 1):
            unroll_op += f"""
            y[i+{j}] += a[i+{j} + k*lda] * tmp_x;"""

        code = f"""#include <stdint.h>
#include <stdio.h>

void mydgemv_n_ver0_1_unroll{unroll}(double a[], double x[], double y[], int64_t lda, int64_t ldx, int64_t ldy){{
    int64_t i, i_remain, counter_lda;
    double tmp_x;
    for (int64_t i=0; i<lda; i++){{
        y[i] = 0.0;
    }}
    
    i_remain = lda % {unroll};
    for (int64_t k=0; k<ldx; k++){{
        tmp_x = x[k];
        i = 0;
        for(int64_t i=0; i<lda-i_remain; i+={unroll}){{
{unroll_op}
        }}
        {remain}
    }}
}}
        """
        f.write(code)
        

        
        # transposed ---------------------------------------------------------------------------------
        remain = ""
        declare_tmp =  "double " + ", ".join([f"tmp{i}" for i in range(unroll)])
        init_tmp = "; ".join([f"tmp{i} = 0.0" for i in range(unroll)])
        if unroll>=2:
            remain = f"""
    for(int64_t i=lda-i_remain; i<lda; i++){{
        tmp0 = 0.0;
        for(int64_t k=0; k<ldx; k++){{
            tmp0 += a_t[k + i*ldx] * x[k];        
        }}
        y[i] = tmp0;    
    }}
            """
            
        store_tmp = ""
        for j in range(0, unroll, 1):
            store_tmp += f"""y[i+{j}] = tmp{j};
            """

        unroll_op = ""
        for j in range(0, unroll, 1):
            unroll_op += f"""tmp{j} += a_t[k + (i+{j})*ldx] * tmp_x;
            """

        code = f"""

void mydgemv_t_ver0_1_unroll{unroll}(double a_t[], double x[], double y[], int64_t lda, int64_t ldx, int64_t ldy){{
    {declare_tmp};
    int64_t i_remain = lda % {unroll};
    double tmp_x;
    
    for(int64_t i=0; i<lda-i_remain; i+={unroll}){{
        {init_tmp};
        for (int64_t k=0; k<ldx; k++){{
            tmp_x = x[k];
            {unroll_op}
        }}
        {store_tmp}
    }}
    {remain}
}}
        """
        f.write(code)        

In [90]:
for i in range(1, 9, 1):
    print(f"""
    subroutine mydgemv_n_ver0_1_unroll{i}(a_t, x, y, lda, ldx, ldy) bind(C, name='mydgemv_n_ver0_1_unroll{i}')
        import
        integer(c_int64_t), value :: lda, ldx, ldy
        real(c_double), intent(in) :: a_t(ldx, lda), x(ldx), y(ldy)
    end subroutine mydgemv_n_ver0_1_unroll{i}

    subroutine mydgemv_t_ver0_1_unroll{i}(a_t, x, y, lda, ldx, ldy) bind(C, name='mydgemv_t_ver0_1_unroll{i}')
        import
        integer(c_int64_t), value :: lda, ldx, ldy
        real(c_double), intent(in) :: a_t(ldx, lda), x(ldx), y(ldy)
    end subroutine mydgemv_t_ver0_1_unroll
    """)


    subroutine mydgemv_n_ver0_1_unroll1(a_t, x, y, lda, ldx, ldy) bind(C, name='mydgemv_n_ver0_1_unroll1')
        import
        integer(c_int64_t), value :: lda, ldx, ldy
        real(c_double), intent(in) :: a_t(ldx, lda), x(ldx), y(ldy)
    end subroutine mydgemv_n_ver0_1_unroll1

    subroutine mydgemv_t_ver0_1_unroll1(a_t, x, y, lda, ldx, ldy) bind(C, name='mydgemv_t_ver0_1_unroll1')
        import
        integer(c_int64_t), value :: lda, ldx, ldy
        real(c_double), intent(in) :: a_t(ldx, lda), x(ldx), y(ldy)
    end subroutine mydgemv_t_ver0_1_unroll
    

    subroutine mydgemv_n_ver0_1_unroll2(a_t, x, y, lda, ldx, ldy) bind(C, name='mydgemv_n_ver0_1_unroll2')
        import
        integer(c_int64_t), value :: lda, ldx, ldy
        real(c_double), intent(in) :: a_t(ldx, lda), x(ldx), y(ldy)
    end subroutine mydgemv_n_ver0_1_unroll2

    subroutine mydgemv_t_ver0_1_unroll2(a_t, x, y, lda, ldx, ldy) bind(C, name='mydgemv_t_ver0_1_unroll2')
        import
        integer